In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import folium
WPG = [49.848695, -97.049665]

In [20]:
from branca.element import Figure
fig=Figure(width=600,height=600)
m1=folium.Map(width=600,height=600,location=WPG,zoom_start=12)
folium.TileLayer('Stamen Terrain').add_to(m1)
folium.TileLayer('Stamen Toner').add_to(m1)
folium.TileLayer('Stamen Water Color').add_to(m1)
folium.TileLayer('cartodbpositron').add_to(m1)
folium.TileLayer('cartodbdark_matter').add_to(m1)
folium.LayerControl().add_to(m1)
m1

In [34]:
fig3=Figure(width=550,height=550)
m3=folium.Map(location=[ 49.934985, -97.096166],tiles='cartodbpositron',zoom_start=11)
fig3.add_child(m3)

folium.Marker(location=[ 49.826952, -97.152312],popup='Default popup Marker1',tooltip='Click here to see Popup').add_to(m3)
folium.Marker(location=[ 49.848695, -97.049665],popup='<strong>Marker3</strong>',tooltip='<strong>Click here to see Popup</strong>').add_to(m3)
folium.Marker(location=[ 49.850579, -97.112202],popup='<h3 style="color:green;">Marker2</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m3)
folium.Marker(location=[ 49.8819,   -97.089342],popup='Default popup Marker1',tooltip='Click here to see Popup').add_to(m3)
folium.Marker(location=[ 49.906744, -97.123028],popup='<strong>Marker3</strong>',tooltip='<strong>Click here to see Popup</strong>').add_to(m3)
folium.Marker(location=[ 49.945949, -97.187758],popup='<h3 style="color:green;">Marker2</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m3)
folium.Marker(location=[ 49.951733, -97.149032],icon=folium.Icon(color='red',icon='x')).add_to(m3)
folium.Marker(location=[ 49.934985, -97.096166]).add_to(m3)
m3

In [ ]:
stations_loc = [[ 49.826952, -97.152312],
 [ 49.848695, -97.049665],
 [ 49.850579, -97.112202],
 [ 49.8819,   -97.089342],
 [ 49.906744, -97.123028],
 [ 49.945949, -97.187758],
 [ 49.951733, -97.149032],
 [ 49.934985, -97.096166]]

In [110]:
def import_data(data):
    def _import_on_time(dataset):
        DF = pd.read_csv(dataset,low_memory=False).drop(columns=["Unnamed: 0"])
        DF["Route"] = [tuple(l) for l in DF["Route"].str.replace("\(|\)|\'","",regex=True).str.split(", ")]
        DF["Scheduled Time"] = pd.to_datetime(DF["Scheduled Time"])
        return DF

    def _import_traffic_counts(dataset):
        DF = pd.read_csv(dataset).drop(columns="Unnamed: 0")
        DF["Timestamp"] = pd.to_datetime(DF["Timestamp"]) 
        return DF

    def _import_sites(dataset):
        return pd.read_csv(dataset)

    def _import_stops(dataset):
        return pd.read_csv(dataset).drop(columns="Unnamed: 0")

    def _import_lane_closure(dataset):
        DF = pd.read_csv(dataset).drop(columns="Unnamed: 0")
        DF["Boundaries"] = DF["Boundaries"].str.replace("\[|\]|\'","",regex=True).str.split(", ")
        DF["Direction"] = DF["Direction"].str.replace("\[|\]|\'","",regex=True).str.split(", ")
        DF["Date Closed - From"] = pd.to_datetime(DF["Date Closed - From"]) 
        DF["Date Closed - To"] = pd.to_datetime(DF["Date Closed - To"]) 
        DF["Geometry"] = [[[(float(l.replace("(","").split(", ")[0]),float(l.replace(")","").split(", ")[1])) for l in s.split("), (")] for s in m] 
                            for m in [j.replace("[[","").replace("]]","").split("], [") for j in DF["Geometry"].values]]
        return DF

    import_dict = {
        "ON_TIME":["clean_datasets/ON_TIME.csv",_import_on_time],
        "TRAFFIC_COUNTS":["clean_datasets/TRAFFIC_COUNTS.csv",_import_traffic_counts],
        "SITES":["clean_datasets/SITES.csv",_import_sites],
        "STOPS":["clean_datasets/STOPS.csv",_import_stops],
        "LANE_CLOSURE":["clean_datasets/LANE_CLOSURE.csv",_import_lane_closure],
    }
    dataset,import_func = import_dict[data]
    return import_func(dataset)

In [111]:
LANE_CLOSURE = import_data("LANE_CLOSURE")
LANE_CLOSURE

,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Bryce,River,"[River Av, End]","[Eastbound, Westbound]",2021-08-16,2021-10-16,12:00 AM,12:00 AM,No,49.879699,-97.142781,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,Lilac,Carter,"[Carter Av, Weatherdon Av]",[Southbound],2021-06-14,2021-10-29,12:00 AM,12:00 AM,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"[Pacific Av, Rupert Av]",[Southbound],2020-08-14,2021-10-16,12:00 AM,12:00 AM,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"[Austin St, End]",[Westbound],2021-05-10,2021-11-30,12:00 AM,11:00 PM,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"[Tache Av, St Joseph St]",[Eastbound],2021-05-31,2022-01-13,12:00 AM,12:00 AM,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Sherbrook,Logan,"[Logan Av, William Av]",[Southbound],2021-09-20,2021-11-01,12:00 AM,12:00 AM,No,49.906584,-97.153705,"[[(49.904179261579, -97.155653416614), (49.904...",1,30
426,Gull Lake,Markham,"[Markham Rd, East Lake Dr]","[Northbound, Southbound]",2021-09-17,2021-10-08,12:00 AM,12:00 AM,No,49.809311,-97.158976,"[[(49.80730904568, -97.158914003228), (49.8081...",1,39
427,Wardlaw,Osborne,"[Osborne St, Scott St]",[Eastbound],2021-09-20,2021-10-15,12:00 AM,12:00 AM,No,49.877534,-97.142232,"[[(49.876961712509, -97.143938755153), (49.877...",1,5
428,Scotsborough,Beckinsale,"[Beckinsale By, Novavista Dr]",[Southbound],2021-09-23,2021-10-13,12:00 AM,12:00 AM,No,49.820905,-97.107388,"[[(49.820524519173, -97.107184511392), (49.820...",1,4


In [112]:
LANE_CLOSURE.sort_values(["Num_Segments"],ascending=False).head(10)

,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
411,Haney,Eldridge,"[Eldridge Av, Huddersfield Pl]",[Northbound],2021-09-03,2021-10-08,12:00 AM,12:00 AM,No,49.852055,-97.263471,"[[(49.852542524467, -97.263561929554), (49.852...",11,243
75,Dugald,Lagimodiere,"[Lagimodiere Bv, Murdock Rd]",[Eastbound],2021-09-06,2021-10-08,12:00 AM,12:00 AM,No,49.885937,-97.026672,"[[(49.885796024272, -97.049499066926), (49.885...",9,101
296,Dugald,Dugald Rd,[At Dugald Rd Overpass],"[Eastbound, Westbound]",2021-10-08,2021-10-08,12:00 AM,12:00 AM,No,49.885912,-97.065875,"[[(49.885980177464, -97.067061900887), (49.885...",4,29
375,Salter,Selkirk,"[Selkirk Av, Slaw Rebchuk Bridge]","[Northbound, Southbound]",2021-08-27,2021-10-15,12:00 AM,12:00 AM,No,49.912269,-97.142885,"[[(49.911834158338, -97.143086768139), (49.911...",4,26
204,Scurfield,Dovercourt,"[Dovercourt Dr, Kenaston Bv]",[Westbound],2021-10-12,2021-10-12,09:00 AM,03:00 PM,No,49.813967,-97.199003,"[[(49.816387083859, -97.194402332154), (49.816...",4,116
369,Summit,Saskatchewan,"[Saskatchewan Av, End]","[Northbound, Southbound]",2021-10-08,2021-10-08,12:00 AM,12:00 AM,Yes,49.905382,-97.294054,"[[(49.899605690143, -97.305727876597), (49.900...",3,213
219,St James,Academy,"[Academy Rd, Century St]",[Northbound],2021-10-16,2021-10-17,12:00 AM,12:00 AM,No,49.877599,-97.202516,"[[(49.874527169756, -97.203518528813), (49.874...",3,98
255,Regent Av,Kanata,"[Kanata St, Day St]",[Westbound],2021-10-06,2021-10-29,12:00 AM,12:00 AM,No,49.894691,-97.002890,"[[(49.895129997181, -97.005484142208), (49.895...",3,28
385,Keewatin,Logan,"[Logan Av, Selkirk Av]","[Northbound, Southbound]",2021-07-27,2021-10-22,12:00 AM,12:00 AM,No,49.926892,-97.195248,"[[(49.921824144979, -97.195915153849), (49.921...",3,61
45,Chalmers,Gateway,"[Gateway Rd, Brazier St]",[Westbound],2021-10-11,2021-10-25,10:00 AM,04:00 PM,No,49.911588,-97.103523,"[[(49.911612097794, -97.102807031379), (49.911...",3,21


In [135]:
fig5=Figure(height=850,width=1250)
m5=folium.Map(location=lane[["Lat","Long"]],tiles='cartodbpositron',zoom_start=15)
fig5.add_child(m5)
f1=folium.FeatureGroup("Lane Closure")
for j in LANE_CLOSURE.index:
    lane = LANE_CLOSURE.iloc[j]
    for i in range(len(lane["Geometry"])):
        line_1=folium.vector_layers.PolyLine(lane["Geometry"][i],popup=f'<b>Complete Closure:</b> {lane["Complete Closure"]}',tooltip=f'{lane["Street"]}',color='orange',weight=4).add_to(f1)
f1.add_to(m5)
m5

In [ ]:
folium.Circle(
    radius=100,
    location=[45.5244, -122.6699],
    popup="The Waterfront",
    color="crimson",
    fill=False,
).add_to(m)

In [96]:
Sherbrook = LANE_CLOSURE.iloc[411]
Sherbrook
fig5=Figure(height=550,width=550)
m5=folium.Map(location=Sherbrook[["Lat","Long"]],tiles='cartodbpositron',zoom_start=14)
fig5.add_child(m5)

f1=folium.FeatureGroup("Vehicle 1")
for i in range(len(Sherbrook["Geometry"])):
    line_1=folium.vector_layers.PolyLine(Sherbrook["Geometry"][i],popup='<b>Closure</b>',tooltip='Sherbrook',color='orange',weight=9).add_to(f1)
f1.add_to(m5)
folium.LayerControl().add_to(m5)
m5

In [97]:
LANE_CLOSURE.iloc[411]

Street                                                            Haney
At                                                             Eldridge
Boundaries                               [Eldridge Av, Huddersfield Pl]
Direction                                                  [Northbound]
Date Closed - From                                  2021-09-03 00:00:00
Date Closed - To                                    2021-10-08 00:00:00
Time Closed - From                                             12:00 AM
Time Closed - To                                               12:00 AM
Complete Closure                                                     No
Lat                                                           49.852055
Long                                                         -97.263471
Geometry              [[(49.852542524467, -97.263561929554), (49.852...
Name: 411, dtype: object